In [1]:
import pandas as pd
import pathlib
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, Float, String, Date, ForeignKey, Sequence
from sqlalchemy.orm import Session

# Create path object using pathlib
This will ensure this file will run for everyone regardless of os
This group contains folks running on mac, windows, and linux.    

In [2]:
cwd = pathlib.Path.cwd()

if cwd.name == 'Project_3_Group_7':
    print("Path is project root")
else:
    print("Please correct current working directory to the project root")

Path is project root


In [3]:
resources_path = pathlib.PurePath(cwd, 'Resources')
resources_path

PurePosixPath('/home/mox/Documents/coding_projects/bootcamp_local/Homeworks/Project_3_Group_7/Resources')

# Prep sqlalchemy for loading data into db

In [4]:
Base = declarative_base()
# tag_table = "tags"
steam_twitch_tags_table = "steam_twitch_tags"

/tmp/ipykernel_116294/2520330224.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [5]:
# class Tag(Base):
#     __tablename__ = tag_table
#     id = Column(Integer, Sequence('user_id_seq'), primary_key=True)
#     app_id = Column(Integer, ForeignKey("combined_twitch_steam.app_id"), nullable=False)
#     tag = Column(String(128))

In [6]:
class combined_twitch_steam_tags(Base):
    __tablename__ = steam_twitch_tags_table
    title = Column(String(128))
    # These two will need to be changed to Float 
    hours_watched = Column(Integer)
    hours_streamed = Column(Integer)
    average_streamers = Column(Integer)
    average_viewers = Column(Integer)
    average_channels = Column(Integer)
    app_id = Column(Integer, primary_key=True)
    release_date = Column(Date)
    reviews_total = Column(Integer)
    review_avg_percent = Column(Integer)
    launch_price_cents = Column(Integer)
    dataset_est_rev_cents = Column(Integer)



In [7]:
engine = sqlalchemy.create_engine(f'sqlite:///{resources_path}/data.db')
Base.metadata.create_all(engine)

# Load pandas data into memory

In [8]:
tag_df = pd.read_csv(f"{resources_path}/app_id_common_tags.csv")
tag_df.head(3)

,app_id,tag
0,730,FPS
1,730,Shooter
2,730,Multiplayer


In [9]:
steam_twitch_df = pd.read_csv(f"{resources_path}/merge_output.csv")
steam_twitch_df.head(3)

,title,hours_watched,hours_streamed,average_streamers,average_viewers,average_channels,app_id,release_date,reviews_total,review_avg_percent,launch_price_cents,dataset_est_rev_cents
0,.hack//G.U. Last Recode,145350,35258,3174,202,49,525480,2017-11-03,2689,87.0,4999,13442311
1,20 Minutes Till Dawn,911356,12253,3616,1267,17,1966900,2023-06-08,20942,92.0,499,10450058
2,60 Parsecs!,529688,1867,606,736,2,646270,2018-09-18,2315,89.0,999,2312685


In [10]:
prep_steam_twitch = steam_twitch_df.set_index('app_id')
large_df = tag_df.join(prep_steam_twitch, on='app_id', how='inner')
large_df

,app_id,tag,title,hours_watched,hours_streamed,average_streamers,average_viewers,average_channels,release_date,reviews_total,review_avg_percent,launch_price_cents,dataset_est_rev_cents
0,730,FPS,Counter-Strike: Global Offensive,4315553263,80709385,123006,66604,1243,2012-08-21,7382695,88.0,1499,11066659805
1,730,Shooter,Counter-Strike: Global Offensive,4315553263,80709385,123006,66604,1243,2012-08-21,7382695,88.0,1499,11066659805
2,730,Multiplayer,Counter-Strike: Global Offensive,4315553263,80709385,123006,66604,1243,2012-08-21,7382695,88.0,1499,11066659805
3,730,Competitive,Counter-Strike: Global Offensive,4315553263,80709385,123006,66604,1243,2012-08-21,7382695,88.0,1499,11066659805
4,730,Action,Counter-Strike: Global Offensive,4315553263,80709385,123006,66604,1243,2012-08-21,7382695,88.0,1499,11066659805
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106366,968870,Horror,Close to the Sun,343108,3465,867,461,4,2020-05-05,506,75.0,1999,1011494
106367,968870,Violent,Close to the Sun,343108,3465,867,461,4,2020-05-05,506,75.0,1999,1011494
106368,968870,Female Protagonist,Close to the Sun,343108,3465,867,461,4,2020-05-05,506,75.0,1999,1011494
106369,968870,First Person,Close to the Sun,343108,3465,867,461,4,2020-05-05,506,75.0,1999,1011494


# Add dataframes to DB

In [11]:
conn = engine.connect()

In [12]:
large_df.to_sql(steam_twitch_tags_table, conn, if_exists='replace')

15403